<a href="https://colab.research.google.com/github/TokyoYoshida/GoogleColab/blob/master/playground_deep_learning_keiba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [382]:
from google.colab import drive
drive.mount('/content/drive')
# 必要ならば以下のようにディレクトリ移動する
 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [383]:
%cd /content/drive/'My Drive'/'★個人専用ファイル'/'開発用・個人サービス開発'/'GoogleColab'/

/content/drive/My Drive/★個人専用ファイル/開発用・個人サービス開発/GoogleColab


In [384]:
%ls data/keiba


keiba_dat_res_20200528_2355.pkl  keiba_dat_res_20200529_2306.pkl
keiba_dat_res_20200528.pkl       result_20200528_2020年時点最新競馬データ.csv


In [0]:
import pandas as pd

In [0]:
import numpy as np

In [0]:
from pprint import pprint

In [0]:
# 行も列もなるべく省略しない
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

In [389]:
!pip install tensorflow==2.2.0

In [0]:
!pip install -q keras

In [0]:
import keras
from keras.utils import np_utils


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,LeakyReLU
from keras.optimizers import RMSprop


In [0]:
# タイム値を秒数(float)に変換する。小数以下１桁まであり
def date_like_value_to_timestamp(df,col_name):
    tmp = df[col_name].str.extract('(.+):(.+)\.(.+)', expand=True)
    tmp.columns=["m","s","dec"]
    return tmp["m"].astype("float")*60+tmp["s"].astype("float")+tmp["dec"].astype("float")*0.1    

In [0]:
# 競馬データの作成

In [0]:
ldat = pd.read_pickle("./data/keiba/keiba_dat_res_20200529_2306.pkl").drop(columns=["index"])

In [0]:
# 前回の3位以内にマークを付ける
ldat["prev_high_rank"] = 0
ldat.loc[ldat["prev_rank"] <= 3, "prev_high_rank"] = 1

In [0]:
# 3位以内にマークを付ける
ldat["high_rank"] = 0
ldat.loc[ldat["rank"] <= 3, "high_rank"] = 1

In [0]:
# 前回ランクを段階化
ldat["prev_rank_level"] = round(ldat["prev_rank"]  / 4)

In [0]:
# ランクを段階化
ldat["rank_level"] = round(ldat["rank"]  / 4)

In [0]:
# 人気を段階化
ldat["prev_popularity_level"] = round(ldat["prev_popularity"] / 4)

In [0]:
# スピードを出す
ldat["prev_speed"] = ldat["prev_distance"] / ldat["prev_time_float"] 

In [0]:
# スピードを段階化
ldat["prev_speed_level"] =  round(ldat["prev_speed"]*10)*0.1

In [0]:
# 学習、テストデータの作成

In [0]:
ddat = ldat.dropna()

In [0]:
trainlen = int(len(ddat)*0.8)

In [0]:
train = ddat.head(trainlen)
test = ddat.tail(len(ddat)-trainlen)


In [0]:
train = ddat.head(trainlen)
test = ddat.tail(len(ddat)-trainlen)


In [0]:
# トレーニングデータは勝ち馬と負け馬をちょうど半々になるようにする start

In [0]:
winner = train.query("prev_high_rank == 1")

In [0]:
loser = train.query("prev_high_rank != 1").head(winner.count()[0])

In [0]:
catted = pd.concat([winner,loser])

In [0]:
# トレーニングデータは勝ち馬と負け馬をちょうど半々になるようにする end

In [0]:
# ランダムサンプリングしてトレーニング、テスト、最終確認用データを作る

In [0]:
train_sampled = catted.sample(frac=1)
test_sampled = test.sample(frac=1)
testorg_sampled = test.sample(frac=1)

In [415]:
test

,age,amount,condition,day,diff_arrival,diff_weight,direction,distance,frame_num,full_path,grade,ground,horse_num,jockey,month,name,odds,popularity,prev_age,prev_amount,prev_condition,prev_day,prev_diff_arrival,prev_diff_weight,prev_direction,prev_distance,prev_frame_num,prev_full_path,prev_grade,prev_ground,prev_horse_num,prev_jockey,prev_month,prev_name,prev_odds,prev_popularity,prev_race_index,prev_race_num,prev_rank,prev_return,prev_sex,prev_start_time,prev_time,prev_time_float,prev_trainer,prev_unknown,prev_weather,prev_weight,prev_winner,prev_year,prev_ymd,race_index,race_num,rank,return,sex,start_time,time,time_float,trainer,unknown,weather,weight,winner,year,ymd,prev_high_rank,high_rank,prev_rank_level,rank_level,prev_popularity_level,prev_speed,prev_speed_level
529772,3,56.0,0,23,0,-2.0,0,2000.0,5,html/201008010710.html,9,1,5,149,1,2838,3.4,2.0,2.0,55.0,0.0,26.0,11.0,-2.0,0.0,2000.0,1.0,html/200909050711.html,3.0,1.0,1.0,149.0,12.0,2838.0,8.1,3.0,39911.0,11.0,4.0,0.0,1.0,15:45,2:01.5,121.5,103.0,-,0.0,462.0,False,2009.0,20091226.0,36900,10,1.0,3.4,1,15:10,2:02.0,122.0,103,-,0,460.0,True,2010,20100123,0,1,1.0,0.0,1.0,16.460905,16.5
529773,3,56.0,0,23,2,2.0,0,2000.0,7,html/201008010710.html,9,1,7,85,1,17385,114.1,7.0,3.0,56.0,0.0,10.0,0.0,10.0,0.0,2000.0,1.0,html/201008010305.html,9.0,1.0,2.0,85.0,1.0,17385.0,11.7,5.0,15715.0,5.0,1.0,11.7,1.0,12:25,2:02.1,122.1,52.0,-,0.0,510.0,True,2010.0,20100110.0,36900,10,4.0,0.0,1,15:10,2:02.8,122.8,52,-,0,512.0,False,2010,20100123,1,0,0.0,1.0,1.0,16.380016,16.4
529774,3,56.0,0,23,6,-6.0,0,2000.0,6,html/201008010710.html,9,1,6,53,1,28949,25.7,5.0,2.0,55.0,0.0,26.0,2.0,4.0,0.0,2000.0,4.0,html/200909050711.html,3.0,1.0,6.0,53.0,12.0,28949.0,85.0,11.0,39911.0,11.0,6.0,0.0,1.0,15:45,2:01.6,121.6,28.0,-,0.0,494.0,False,2009.0,20091226.0,36900,10,3.0,0.0,1,15:10,2:02.8,122.8,28,-,0,488.0,False,2010,20100123,0,1,2.0,1.0,3.0,16.447368,16.4
529775,3,56.0,0,23,14,-10.0,0,2000.0,2,html/201008010710.html,9,1,2,0,1,1623,1.7,1.0,2.0,55.0,0.0,27.0,0.0,0.0,0.0,2000.0,1.0,html/200909050806.html,9.0,1.0,1.0,0.0,12.0,1623.0,1.5,1.0,14328.0,6.0,1.0,1.5,1.0,12:40,2:04.7,124.7,55.0,-,1.0,498.0,True,2009.0,20091227.0,36900,10,2.0,0.0,1,15:10,2:02.2,122.2,55,-,0,488.0,False,2010,20100123,1,1,0.0,0.0,0.0,16.038492,16.0
529776,3,56.0,0,23,5,-6.0,0,2000.0,3,html/201008010710.html,9,1,3,45,1,14752,16.3,4.0,2.0,55.0,0.0,27.0,15.0,4.0,0.0,1600.0,5.0,html/200909050807.html,9.0,1.0,10.0,45.0,12.0,14752.0,2.3,1.0,20318.0,7.0,2.0,0.0,1.0,13:10,1:35.1,95.1,7.0,外,0.0,496.0,False,2009.0,20091227.0,36900,10,5.0,0.0,1,15:10,2:03.0,123.0,7,-,0,490.0,False,2010,20100123,1,0,0.0,1.0,0.0,16.824395,16.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661567,2,54.0,0,7,14,0.0,0,1800.0,8,html/201102020801.html,9,1,11,54,8,12661,1.6,1.0,2.0,54.0,0.0,9.0,16.0,0.0,0.0,1800.0,7.0,html/201102010705.html,9.0,1.0,8.0,54.0,7.0,12661.0,42.5,9.0,22006.0,5.0,2.0,0.0,1.0,11:50,1:51.2,111.2,146.0,-,0.0,456.0,False,2011.0,20110709.0,46077,1,2.0,0.0,1,09:55,1:52.5,112.5,146,-,0,456.0,False,2011,20110807,1,1,0.0,0.0,2.0,16.187050,16.2
661568,2,54.0,0,7,0,-8.0,0,1800.0,1,html/201102020801.html,9,1,1,24,8,10763,24.1,6.0,2.0,54.0,0.0,31.0,14.0,0.0,0.0,1800.0,8.0,html/201102020605.html,9.0,1.0,14.0,24.0,7.0,10763.0,10.8,4.0,14400.0,5.0,5.0,0.0,1.0,11:50,1:53.4,113.4,187.0,-,0.0,476.0,False,2011.0,20110731.0,46077,1,1.0,24.1,1,09:55,1:52.3,112.3,187,-,0,468.0,True,2011,20110807,0,1,1.0,0.0,1.0,15.873016,15.9
661569,2,54.0,0,7,4,-4.0,0,1800.0,5,html/201102020801.html,9,1,5,105,8,36039,163.5,11.0,2.0,54.0,0.0,9.0,6.0,0.0,0.0,1800.0,2.0,html/201102010705.html,9.0,1.0,2.0,105.0,7.0,36039.0,16.7,8.0,22006.0,5.0,7.0,0.0,0.0,11:50,1:53.2,113.2,16.0,-,0.0,490.0,False,2011.0,20110709.0,46077,1,11.0,0.0,0,09:55,1:5

In [416]:
testorg_sampled.query("race_index == 36900")

,age,amount,condition,day,diff_arrival,diff_weight,direction,distance,frame_num,full_path,grade,ground,horse_num,jockey,month,name,odds,popularity,prev_age,prev_amount,prev_condition,prev_day,prev_diff_arrival,prev_diff_weight,prev_direction,prev_distance,prev_frame_num,prev_full_path,prev_grade,prev_ground,prev_horse_num,prev_jockey,prev_month,prev_name,prev_odds,prev_popularity,prev_race_index,prev_race_num,prev_rank,prev_return,prev_sex,prev_start_time,prev_time,prev_time_float,prev_trainer,prev_unknown,prev_weather,prev_weight,prev_winner,prev_year,prev_ymd,race_index,race_num,rank,return,sex,start_time,time,time_float,trainer,unknown,weather,weight,winner,year,ymd,prev_high_rank,high_rank,prev_rank_level,rank_level,prev_popularity_level,prev_speed,prev_speed_level
529773,3,56.0,0,23,2,2.0,0,2000.0,7,html/201008010710.html,9,1,7,85,1,17385,114.1,7.0,3.0,56.0,0.0,10.0,0.0,10.0,0.0,2000.0,1.0,html/201008010305.html,9.0,1.0,2.0,85.0,1.0,17385.0,11.7,5.0,15715.0,5.0,1.0,11.7,1.0,12:25,2:02.1,122.1,52.0,-,0.0,510.0,True,2010.0,20100110.0,36900,10,4.0,0.0,1,15:10,2:02.8,122.8,52,-,0,512.0,False,2010,20100123,1,0,0.0,1.0,1.0,16.380016,16.4
529776,3,56.0,0,23,5,-6.0,0,2000.0,3,html/201008010710.html,9,1,3,45,1,14752,16.3,4.0,2.0,55.0,0.0,27.0,15.0,4.0,0.0,1600.0,5.0,html/200909050807.html,9.0,1.0,10.0,45.0,12.0,14752.0,2.3,1.0,20318.0,7.0,2.0,0.0,1.0,13:10,1:35.1,95.1,7.0,外,0.0,496.0,False,2009.0,20091227.0,36900,10,5.0,0.0,1,15:10,2:03.0,123.0,7,-,0,490.0,False,2010,20100123,1,0,0.0,1.0,0.0,16.824395,16.8
529774,3,56.0,0,23,6,-6.0,0,2000.0,6,html/201008010710.html,9,1,6,53,1,28949,25.7,5.0,2.0,55.0,0.0,26.0,2.0,4.0,0.0,2000.0,4.0,html/200909050711.html,3.0,1.0,6.0,53.0,12.0,28949.0,85.0,11.0,39911.0,11.0,6.0,0.0,1.0,15:45,2:01.6,121.6,28.0,-,0.0,494.0,False,2009.0,20091226.0,36900,10,3.0,0.0,1,15:10,2:02.8,122.8,28,-,0,488.0,False,2010,20100123,0,1,2.0,1.0,3.0,16.447368,16.4
529772,3,56.0,0,23,0,-2.0,0,2000.0,5,html/201008010710.html,9,1,5,149,1,2838,3.4,2.0,2.0,55.0,0.0,26.0,11.0,-2.0,0.0,2000.0,1.0,html/200909050711.html,3.0,1.0,1.0,149.0,12.0,2838.0,8.1,3.0,39911.0,11.0,4.0,0.0,1.0,15:45,2:01.5,121.5,103.0,-,0.0,462.0,False,2009.0,20091226.0,36900,10,1.0,3.4,1,15:10,2:02.0,122.0,103,-,0,460.0,True,2010,20100123,0,1,1.0,0.0,1.0,16.460905,16.5
529775,3,56.0,0,23,14,-10.0,0,2000.0,2,html/201008010710.html,9,1,2,0,1,1623,1.7,1.0,2.0,55.0,0.0,27.0,0.0,0.0,0.0,2000.0,1.0,html/200909050806.html,9.0,1.0,1.0,0.0,12.0,1623.0,1.5,1.0,14328.0,6.0,1.0,1.5,1.0,12:40,2:04.7,124.7,55.0,-,1.0,498.0,True,2009.0,20091227.0,36900,10,2.0,0.0,1,15:10,2:02.2,122.2,55,-,0,488.0,False,2010,20100123,1,1,0.0,0.0,0.0,16.038492,16.0


In [0]:
# xデータ、yデータに振り分ける

In [0]:
#  x = test_sampled[['frame_num', 'horse_num', 'name', 'sex', 'age', 'amount',
#        'jockey', 'odds', 'popularity', 'weight',
#        'diff_weight', 'trainer', 'race_num', 'grade', 'ground', 'direction',
#         'distance', 'weather', 'condition', 'start_time_float', 'year',
#        'month', 'day', 'prev_rank', 'prev_frame_num', 'prev_horse_num',
#        'prev_name', 'prev_sex', 'prev_age', 'prev_amount', 'prev_jockey',
#        'prev_time_float', 'prev_diff_arrival', 'prev_odds', 'prev_popularity',
#        'prev_weight', 'prev_diff_weight', 'prev_trainer', 'prev_race_num',
#        'prev_grade', 'prev_ground', 'prev_direction',
#        'prev_distance', 'prev_weather', 'prev_condition', 'prev_start_time_float',
#        'prev_year', 'prev_month', 'prev_day']].dropna(0)

In [0]:
# x = sdat[['frame_num', 'horse_num', 'sex', 'age', 'amount',
#        'odds', 'popularity', 'weight',
#        'diff_weight', 'trainer', 'grade', 'ground', 'direction',
#         'distance', 'weather', 'condition', 'start_time_float', 'prev_rank', 'prev_frame_num', 'prev_horse_num',
#        'prev_sex', 'prev_age', 'prev_amount', 'prev_jockey',
#        'prev_time_float', 'prev_diff_arrival', 'prev_odds', 'prev_popularity',
#        'prev_weight', 'prev_diff_weight', 'prev_trainer',
#        'prev_grade', 'prev_ground', 'prev_direction',
#        'prev_distance', 'prev_weather', 'prev_condition', 'prev_start_time_float'
#        ]].dropna(0)

In [0]:
# x = sdat[["odds",'prev_rank','prev_diff_arrival']].dropna(0)

,prev_rank_level,prev_rank
0,2.0,9.0
1,2.0,10.0
2,4.0,14.0
3,3.0,11.0
4,2.0,6.0
...,...,...
661567,0.0,2.0
661568,1.0,5.0
661569,2.0,7.0
661570,1.0,5.0


In [0]:
cols = ['prev_rank',"prev_speed_level"]
trainx = train_sampled[cols]
testx = test_sampled[cols]
lastx = testorg_sampled[cols]

In [0]:
cols=["high_rank"]
y_np = np_utils.to_categorical(train_sampled[cols])
trainy = pd.DataFrame(y_np,columns=["lose","win"],index=train_sampled.index)

testy_np = np_utils.to_categorical(test_sampled[cols])
testy = pd.DataFrame(testy_np,columns=["lose","win"],index=test_sampled.index)

lasty_np = np_utils.to_categorical(testorg_sampled[cols])
lasty = pd.DataFrame(lasty_np,columns=["lose","win"],index=testorg_sampled.index)

In [588]:
testy

,lose,win
533921,1.0,0.0
632440,0.0,1.0
547740,1.0,0.0
580038,1.0,0.0
542356,1.0,0.0
...,...,...
569674,1.0,0.0
531315,1.0,0.0
531798,1.0,0.0
607899,1.0,0.0


In [589]:
test_sampled.query("prev_ymd == 20140831.0")

,age,amount,condition,day,diff_arrival,diff_weight,direction,distance,frame_num,full_path,grade,ground,horse_num,jockey,month,name,odds,popularity,prev_age,prev_amount,prev_condition,prev_day,prev_diff_arrival,prev_diff_weight,prev_direction,prev_distance,prev_frame_num,prev_full_path,prev_grade,prev_ground,prev_horse_num,prev_jockey,prev_month,prev_name,prev_odds,prev_popularity,prev_race_index,prev_race_num,prev_rank,prev_return,prev_sex,prev_start_time,prev_time,prev_time_float,prev_trainer,prev_unknown,prev_weather,prev_weight,prev_winner,prev_year,prev_ymd,race_index,race_num,rank,return,sex,start_time,time,time_float,trainer,unknown,weather,weight,winner,year,ymd,prev_high_rank,high_rank,prev_rank_level,rank_level,prev_popularity_level,prev_speed,prev_speed_level
629766,2,54.0,0,28,11,-6.0,1,1800.0,5,html/201404030601.html,9,0,9,103,9,31076,17.5,7.0,2.0,54.0,0.0,31.0,4.0,0.0,0.0,1700.0,2.0,html/201401020405.html,9.0,0.0,2.0,20.0,8.0,31076.0,36.5,7.0,20046.0,5.0,4.0,0.0,1.0,12:10,1:50.3,110.3,128.0,-,0.0,474.0,False,2014.0,20140831.0,43860,1,13.0,0.0,1,10:10,1:57.8,117.8,128,-,0,468.0,False,2014,20140928,0,0,1.0,3.0,2.0,15.412511,15.4
628374,5,55.0,0,18,14,-7.0,0,1400.0,5,html/201509020712.html,9,0,10,8,4,2876,103.4,14.0,4.0,55.0,0.0,31.0,5.0,10.0,0.0,1700.0,7.0,html/201410021008.html,9.0,0.0,13.0,46.0,8.0,2876.0,112.7,14.0,211.0,8.0,15.0,0.0,0.0,13:50,1:49.2,109.2,0.0,-,0.0,492.0,False,2014.0,20140831.0,43760,12,5.0,0.0,0,16:10,1:24.8,84.8,0,-,0,474.0,False,2015,20150418,0,0,4.0,1.0,4.0,15.567766,15.6
643187,4,55.0,0,27,2,-2.0,1,2000.0,6,html/201404030512.html,9,1,11,58,9,12279,14.9,8.0,4.0,55.0,0.0,31.0,15.0,-23.0,1.0,2000.0,4.0,html/201404021006.html,9.0,1.0,5.0,58.0,8.0,12279.0,17.3,5.0,44219.0,6.0,4.0,0.0,0.0,13:00,1:59.7,119.7,200.0,外,0.0,460.0,False,2014.0,20140831.0,44799,12,9.0,0.0,0,16:25,1:59.7,119.7,200,外,0,458.0,False,2014,20140927,0,0,1.0,2.0,1.0,16.708438,16.7
560452,3,51.0,0,5,21,-6.0,0,1200.0,7,html/201409040806.html,9,0,13,83,10,3656,21.3,9.0,3.0,52.0,0.0,31.0,16.0,4.0,0.0,1200.0,3.0,html/201401020407.html,9.0,1.0,3.0,107.0,8.0,3656.0,13.1,4.0,28934.0,7.0,10.0,0.0,0.0,13:10,1:10.2,70.2,15.0,-,0.0,442.0,False,2014.0,20140831.0,39037,6,15.0,0.0,0,12:40,1:14.9,74.9,15,-,1,436.0,False,2014,20141005,0,0,2.0,4.0,1.0,17.094017,17.1
540070,4,57.0,0,27,0,-2.0,1,1800.0,2,html/201404030509.html,9,0,3,71,9,10132,5.0,2.0,4.0,57.0,0.0,31.0,2.0,-8.0,0.0,1700.0,8.0,html/201401020410.html,9.0,0.0,12.0,19.0,8.0,10132.0,2.5,1.0,27631.0,10.0,4.0,0.0,1.0,14:50,1:46.7,106.7,157.0,-,0.0,486.0,False,2014.0,20140831.0,37615,9,1.0,5.0,1,14:35,1:51.5,111.5,157,-,0,484.0,True,2014,20140927,0,1,1.0,0.0,0.0,15.932521,15.9
581600,3,54.0,0,21,16,6.0,0,2200.0,4,html/201409040407.html,9,1,4,11,9,8076,36.1,9.0,3.0,54.0,0.0,31.0,7.0,-6.0,0.0,2000.0,7.0,html/201410021009.html,9.0,1.0,8.0,11.0,8.0,8076.0,103.3,7.0,11171.0,9.0,5.0,0.0,1.0,14:25,2:00.1,120.1,127.0,-,1.0,444.0,False,2014.0,20140831.0,40507,7,4.0,0.0,1,13:15,2:14.8,134.8,127,-,0,450.0,False,2014,20140921,0,0,1.0,1.0,2.0,16.652789,16.7
556833,3,53.0,0,20,7,6.0,0,1200.0,8,html/201409040312.html,9,1,16,14,9,37773,3.3,1.0,3.0,54.0,0.0,31.0,0.0,6.0,0.0,1200.0,2.0,html/201410021006.html,9.0,1.0,4.0,14.0,8.0,37773.0,13.9,6.0,40265.0,6.0,1.0,13.9,0.0,12:50,1:07.7,67.7,157.0,-,0.0,460.0,True,2014.0,20140831.0,38781,12,11.0,0.0,0,16:10,1:09.5,69.5,157,-,1,466.0,False,2014,20140920,1,0,0.0,3.0,2.0,17.725258,17.7
660661,7,57.0,0,18,11,10.0,0,2000.0,6,html/201403030108.html,9,1,12,117,10,2740,46.1,13.0,7.0,57.0,0.0,31.0,5.0,-4.0,0.0,2000.0,7.0,html/201401020409.html,9.0,1.0,11.0,55.0,8.0,2740.0,78.9,14.0,22736.0,9.0,7.0,0.0,2.0,14:15,2:03.6,123.6,123.0,-,0.0,466.0,False,2014.0,20140831.0,46014,8,8.0,0.0,2,13:45,1:59.2,119.2,123,-,0,476.0,False,2014,20141018,0,0,2.0,2.0,4.0,16.181230,16.2
598174,3,54.0,0,13,16,8.0,0,1600.0,2,html/201409040104.html,9,1,4,51,9,61976,12.8,6.0,3.0,54.0,0.0,31.0,6.0,2.0,0.0,1200.0,7.0,html/201410021006.html,9.0,1.0,13.0,299.0,8.0,61976.0,50.9,11.0,40265.0,6.0,3.0,

In [0]:
# 競馬データの作成 終わり

In [0]:
# 学習、テストデータの作成　終わり

In [0]:
#!pip uninstall -y keras

In [0]:
#!pip uninstall -y tensorflow

In [0]:
batch_size = 128
num_classes = 2
epochs = 2


In [0]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(trainx.shape[1],)))
model.add(LeakyReLU(alpha=0.3))

model.add(Dense(256, activation=None))
model.add(LeakyReLU(alpha=0.3))

model.add(Dense(128, activation=None))
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(2)) # 出力層2ノード,全結合
model.add(Activation("sigmoid"))

In [596]:
model.summary()


Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_33 (Dense)             (None, 512)               1536      
_________________________________________________________________
leaky_re_lu_25 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_34 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_26 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 128)               32896     
_________________________________________________________________
leaky_re_lu_27 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_36 (Dense)             (None, 2)               

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])


In [598]:
history = model.fit(trainx, trainy,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(testx, testy))


Train on 204946 samples, validate on 111249 samples
Epoch 1/2
204946/204946 [==============================] - 15s 72us/step - loss: 0.5426 - accuracy: 0.7238 - val_loss: 0.4817 - val_accuracy: 0.7867
Epoch 2/2
204946/204946 [==============================] - 16s 76us/step - loss: 0.5391 - accuracy: 0.7245 - val_loss: 0.4735 - val_accuracy: 0.7867


In [599]:
testy

,lose,win
533921,1.0,0.0
632440,0.0,1.0
547740,1.0,0.0
580038,1.0,0.0
542356,1.0,0.0
...,...,...
569674,1.0,0.0
531315,1.0,0.0
531798,1.0,0.0
607899,1.0,0.0


In [0]:
# 予測
predictions = model.predict(lastx)

In [601]:
print(predictions)

[[0.7589781  0.23804545]
 [0.84247005 0.15272579]
 [0.6091397  0.38928384]
 ...
 [0.9100238  0.08392194]
 [0.7364105  0.25808948]
 [0.783123   0.21202658]]


In [602]:
# 評価
score = model.evaluate(lastx, lasty, verbose=1)


111249/111249 [==============================] - 4s 37us/step


In [603]:
len(lastx)

111249

In [604]:

print("Test score", score[0])
print("Test accuracy", score[1])

Test score 0.47353886430923103
Test accuracy 0.7866677641868591


In [0]:
predict = pd.DataFrame(predictions, columns=["lose","win"] ,index=lastx.index)

In [0]:
result =  pd.concat([ predict,testorg_sampled],axis=1)

In [607]:
predictions

array([[0.7589781 , 0.23804545],
       [0.84247005, 0.15272579],
       [0.6091397 , 0.38928384],
       ...,
       [0.9100238 , 0.08392194],
       [0.7364105 , 0.25808948],
       [0.783123  , 0.21202658]], dtype=float32)

In [608]:
predict

,lose,win
529889,0.758978,0.238045
660527,0.842470,0.152726
544421,0.609140,0.389284
603748,0.901084,0.093721
647825,0.537237,0.461706
...,...,...
540816,0.534002,0.465098
622539,0.537237,0.461706
566820,0.910024,0.083922
571857,0.736410,0.258089


In [609]:
testorg_sampled

,age,amount,condition,day,diff_arrival,diff_weight,direction,distance,frame_num,full_path,grade,ground,horse_num,jockey,month,name,odds,popularity,prev_age,prev_amount,prev_condition,prev_day,prev_diff_arrival,prev_diff_weight,prev_direction,prev_distance,prev_frame_num,prev_full_path,prev_grade,prev_ground,prev_horse_num,prev_jockey,prev_month,prev_name,prev_odds,prev_popularity,prev_race_index,prev_race_num,prev_rank,prev_return,prev_sex,prev_start_time,prev_time,prev_time_float,prev_trainer,prev_unknown,prev_weather,prev_weight,prev_winner,prev_year,prev_ymd,race_index,race_num,rank,return,sex,start_time,time,time_float,trainer,unknown,weather,weight,winner,year,ymd,prev_high_rank,high_rank,prev_rank_level,rank_level,prev_popularity_level,prev_speed,prev_speed_level
529889,2,54.0,2,15,2,0.0,0,1200.0,8,html/201503030610.html,9,1,16,83,11,7587,17.9,6.0,2.0,54.0,0.0,17.0,0.0,-2.0,0.0,1200.0,2.0,html/201508040401.html,9.0,0.0,2.0,3.0,10.0,7587.0,1.3,1.0,11221.0,1.0,1.0,1.3,0.0,10:05,1:13.8,73.8,110.0,-,0.0,436.0,True,2015.0,20151017.0,36908,10,9.0,0.0,0,14:40,1:12.0,72.0,110,-,0,436.0,False,2015,20151115,1,0,0.0,2.0,0.0,16.260163,16.3
660527,5,56.0,0,4,18,4.0,0,1800.0,8,html/201506040912.html,9,0,16,55,10,8592,31.3,10.0,5.0,57.0,0.0,13.0,2.0,-2.0,0.0,1800.0,8.0,html/201509040210.html,9.0,1.0,14.0,12.0,9.0,8592.0,29.1,9.0,30374.0,10.0,8.0,0.0,1.0,15:01,1:46.7,106.7,43.0,外,0.0,498.0,False,2015.0,20150913.0,46005,12,2.0,0.0,1,16:25,1:52.5,112.5,43,-,0,502.0,False,2015,20151004,0,1,2.0,0.0,2.0,16.869728,16.9
544421,4,55.0,0,20,2,12.0,0,1800.0,3,html/201509030508.html,9,1,6,3,6,13989,17.3,8.0,4.0,55.0,0.0,24.0,7.0,-8.0,1.0,1800.0,6.0,html/201504010808.html,9.0,1.0,12.0,147.0,5.0,13989.0,26.9,9.0,41823.0,8.0,3.0,0.0,0.0,13:30,1:47.2,107.2,129.0,外,0.0,448.0,False,2015.0,20150524.0,37920,8,5.0,0.0,0,13:50,1:47.5,107.5,129,外,1,460.0,False,2015,20150620,1,0,1.0,1.0,2.0,16.791045,16.8
603748,2,53.0,0,21,14,-4.0,0,1400.0,1,html/200908050502.html,9,1,1,107,11,10636,136.1,12.0,2.0,54.0,0.0,31.0,11.0,-2.0,0.0,1600.0,8.0,html/200908040702.html,9.0,1.0,12.0,29.0,10.0,10636.0,31.7,7.0,40225.0,2.0,11.0,0.0,0.0,10:30,1:35.9,95.9,179.0,-,0.0,422.0,False,2009.0,20091031.0,42053,2,17.0,0.0,0,10:30,1:24.2,84.2,179,-,0,418.0,False,2009,20091121,0,0,3.0,4.0,2.0,16.684046,16.7
647825,4,57.0,0,2,4,-6.0,1,1300.0,6,html/202005020312.html,9,0,12,139,5,18779,2.2,1.0,4.0,57.0,0.0,4.0,2.0,0.0,0.0,1200.0,2.0,html/202006030312.html,9.0,0.0,4.0,139.0,4.0,18779.0,7.2,5.0,4023.0,12.0,2.0,0.0,1.0,16:25,1:11.1,71.1,239.0,-,0.0,522.0,False,2020.0,20200404.0,45123,12,2.0,0.0,1,16:25,1:17.7,77.7,239,-,0,516.0,False,2020,20200502,1,1,0.0,0.0,1.0,16.877637,16.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540816,3,53.0,0,14,2,-4.0,0,2000.0,6,html/201803020504.html,9,1,11,142,7,42508,9.2,6.0,3.0,52.0,0.0,26.0,2.0,-2.0,1.0,1800.0,1.0,html/201805021103.html,9.0,1.0,1.0,142.0,5.0,42508.0,60.7,8.0,10184.0,3.0,2.0,0.0,0.0,11:10,1:48.5,108.5,217.0,-,1.0,436.0,False,2018.0,20180526.0,37667,4,7.0,0.0,0,11:45,2:02.0,122.0,217,-,0,432.0,False,2018,20180714,1,0,0.0,2.0,2.0,16.589862,16.6
622539,7,56.0,2,14,2,2.0,0,2200.0,7,html/201608020611.html,2,1,12,70,2,16565,32.7,10.0,6.0,57.0,0.0,12.0,14.0,12.0,0.0,1800.0,5.0,html/201509050311.html,3.0,1.0,10.0,5.0,12.0,16565.0,5.0,2.0,2314.0,11.0,2.0,0.0,1.0,15:45,1:46.3,106.3,141.0,外,1.0,454.0,False,2015.0,20151212.0,43354,11,4.0,0.0,1,15:35,2:18.3,138.3,141,外,0,456.0,False,2016,20160214,1,0,0.0,1.0,0.0,16.933208,16.9
566820,3,54.0,4,29,1,8.0,0,1200.0,1,html/202006030202.html,9,0,1,146,3,60227,88.6,12.0,2.0,55.0,0.0,14.0,13.0,0.0,0.0,1800.0,4.0,html/201906050506.html,9.0,0.0,5.0,21.0,12.0,60227.0,23.6,5.0,17666.0,6.0,12.0,0.0,1.0,12:45,2:01.7,121.7,32.0,-,0.0,508.0,False,2019.0,20191214.0,39479,2,

In [0]:
result["final_return"] = result["return"]
result.loc[result["return"] == 0, "final_return"] = -1


In [0]:
# 購入する馬の決定

In [0]:
result["luck"] = result["win"]/ (1/result["odds"])

In [711]:
result.query("win > 0.1")[["win","odds","luck"]]

,win,odds,luck
529889,0.238045,17.9,4.261014
660527,0.152726,31.3,4.780317
544421,0.389284,17.3,6.734610
647825,0.461706,2.2,1.015752
604697,0.468212,2.3,1.076889
...,...,...,...
542067,0.304551,64.7,19.704454
540816,0.465098,9.2,4.278900
622539,0.461706,32.7,15.097770
571857,0.258089,6.8,1.755008


In [0]:
# 単純に勝率が高い馬を買うパターン
predict_win = result.query("win > 0.42 and luck >  5 and odds >= 2")

In [0]:
# そのレースで一番勝率の高い馬を買うパターン
# predict_win = result.loc[result.groupby("race_index")["win"].idxmax()]

In [869]:
len(predict_win)

1342

In [870]:
predict_win[["odds"]].head(5)

,odds
538651,18.5
561201,11.2
605129,43.0
577624,16.1
660405,38.2


In [0]:

return_amount = predict_win.sum()["return"]
highrank_amount = predict_win.query("high_rank == 1").count()["name"]
count_amount = predict_win.count()["name"]

In [872]:
# 回収率
return_amount / count_amount

0.7298062593144561

In [873]:
# 的中率
highrank_amount / count_amount

0.1721311475409836

In [816]:
# 回収額
return_amount - count_amount

-101.69999999999999

In [688]:
result.query("prev_ymd == 20140831.0 and prev_race_num == 11.0")

,lose,win,age,amount,condition,day,diff_arrival,diff_weight,direction,distance,frame_num,full_path,grade,ground,horse_num,jockey,month,name,odds,popularity,prev_age,prev_amount,prev_condition,prev_day,prev_diff_arrival,prev_diff_weight,prev_direction,prev_distance,prev_frame_num,prev_full_path,prev_grade,prev_ground,prev_horse_num,prev_jockey,prev_month,prev_name,prev_odds,prev_popularity,prev_race_index,prev_race_num,prev_rank,prev_return,prev_sex,prev_start_time,prev_time,prev_time_float,prev_trainer,prev_unknown,prev_weather,prev_weight,prev_winner,prev_year,prev_ymd,race_index,race_num,rank,return,sex,start_time,time,time_float,trainer,unknown,weather,weight,winner,year,ymd,prev_high_rank,high_rank,prev_rank_level,rank_level,prev_popularity_level,prev_speed,prev_speed_level,final_return
642433,0.539699,0.459102,2,55.0,0,24,2,4.0,1,1800.0,2,html/201405050711.html,3,1,2,72,11,35500,2.4,1.0,2.0,54.0,0.0,31.0,11.0,6.0,1.0,1600.0,4.0,html/201404021011.html,3.0,1.0,7.0,72.0,8.0,35500.0,3.2,1.0,36906.0,11.0,2.0,0.0,1.0,15:45,1:33.4,93.4,82.0,外,0.0,494.0,False,2014.0,20140831.0,44744,11,2.0,0.0,1,15:25,1:47.9,107.9,82,-,0,498.0,False,2014,20141124,1,1,0.0,0.0,0.0,17.130621,17.1,-1.0
583509,0.606554,0.391862,7,58.0,0,5,14,4.0,0,1600.0,6,html/201409040811.html,9,1,8,11,10,19694,49.2,11.0,7.0,58.0,0.0,31.0,2.0,4.0,0.0,1800.0,7.0,html/201410021011.html,9.0,1.0,10.0,11.0,8.0,19694.0,34.8,10.0,41534.0,11.0,3.0,0.0,1.0,15:35,1:45.6,105.6,203.0,-,1.0,522.0,False,2014.0,20140831.0,40640,11,12.0,0.0,1,15:30,1:34.7,94.7,203,外,1,526.0,False,2014,20141005,1,0,1.0,3.0,2.0,17.045455,17.0,-1.0
553210,0.943021,0.052144,2,55.0,0,9,12,4.0,1,2000.0,7,html/201405050209.html,9,1,7,58,11,44140,62.7,9.0,2.0,54.0,0.0,31.0,17.0,0.0,1.0,1600.0,7.0,html/201404021011.html,3.0,1.0,13.0,58.0,8.0,44140.0,51.7,9.0,36906.0,11.0,15.0,0.0,1.0,15:45,1:36.0,96.0,27.0,外,0.0,426.0,False,2014.0,20140831.0,38533,9,8.0,0.0,1,14:25,2:02.7,122.7,27,-,1,430.0,False,2014,20141109,0,0,4.0,2.0,2.0,16.666667,16.7,-1.0
636777,0.775949,0.222714,3,56.0,0,22,2,2.0,0,1800.0,6,html/201506020811.html,2,1,7,33,3,45556,14.9,6.0,2.0,54.0,0.0,31.0,0.0,8.0,1.0,1600.0,2.0,html/201404021011.html,3.0,1.0,4.0,33.0,8.0,45556.0,4.1,3.0,36906.0,11.0,1.0,4.1,1.0,15:45,1:33.4,93.4,154.0,外,0.0,474.0,True,2014.0,20140831.0,44349,11,7.0,0.0,1,15:45,1:49.6,109.6,154,-,0,476.0,False,2015,20150322,1,0,0.0,2.0,1.0,17.130621,17.1,-1.0
660991,0.814461,0.180636,3,54.0,2,5,5,6.0,1,1200.0,3,html/201507030212.html,9,0,5,81,7,45148,34.1,9.0,2.0,54.0,0.0,31.0,5.0,16.0,1.0,1600.0,2.0,html/201404021011.html,3.0,1.0,3.0,102.0,8.0,45148.0,59.7,10.0,36906.0,11.0,7.0,0.0,1.0,15:45,1:34.3,94.3,6.0,外,0.0,512.0,False,2014.0,20140831.0,46037,12,13.0,0.0,1,16:15,1:12.8,72.8,6,-,1,518.0,False,2015,20150705,0,0,2.0,3.0,2.0,16.967126,17.0,-1.0
598647,0.687526,0.307437,2,55.0,0,2,0,0.0,0,1800.0,3,html/201403030610.html,9,1,3,110,11,32454,2.2,1.0,2.0,54.0,0.0,31.0,5.0,0.0,1.0,1600.0,5.0,html/201404021011.html,3.0,1.0,10.0,110.0,8.0,32454.0,64.9,12.0,36906.0,11.0,4.0,0.0,1.0,15:45,1:33.8,93.8,135.0,外,0.0,478.0,False,2014.0,20140831.0,41695,10,1.0,2.2,1,14:40,1:50.4,110.4,135,-,0,478.0,True,2014,20141102,0,1,1.0,0.0,3.0,17.057569,17.1,2.2
575399,0.866985,0.128396,5,56.0,0,2,2,-4.0,0,1800.0,4,html/201408040911.html,9,1,6,66,11,2220,106.9,13.0,5.0,56.0,0.0,31.0,5.0,-4.0,0.0,1800.0,5.0,html/201410021011.html,9.0,1.0,6.0,14.0,8.0,2220.0,14.4,8.0,41534.0,11.0,9.0,0.0,2.0,15:35,1:46.5,106.5,225.0,-,1.0,482.0,False,2014.0,20140831.0,40073,11,7.0,0.0,2,15:30,1:47.7,107.7,225,外,1,478.0,False,2014,20141102,0,0,2.0,2.0,2.0,16.901408,16.9,-1.0
583511,0.842470,0.152726,7,56.0,0,5,15,24.0,0,1600.0,7,html/201409040811.html,9,1,11,25,10,1447,26.3,9.0,7.0,56.0,0.0,31.0,1.0,-10.0,0.0,1800.0,1.0,html/201410021011.html,9.0,1.0,1.0,13.0,8.0,1447.0,6.4,3.0,41534.0,11.0,8.0,0.0,2.0,15:35,1:46.3,106.3,125.0,-,1.0,524.0,False,2014.0,20140831.0,40640,11,10.0,0.0,2,15:30,1:34.4,94.4,125,外,1,548.0,False,2014,20141005,0,0,2.0,2.0,1.0,16.933208,16.9,-1.0
579714,

In [646]:
predict_win

,lose,win,age,amount,condition,day,diff_arrival,diff_weight,direction,distance,frame_num,full_path,grade,ground,horse_num,jockey,month,name,odds,popularity,prev_age,prev_amount,prev_condition,prev_day,prev_diff_arrival,prev_diff_weight,prev_direction,prev_distance,prev_frame_num,prev_full_path,prev_grade,prev_ground,prev_horse_num,prev_jockey,prev_month,prev_name,prev_odds,prev_popularity,prev_race_index,prev_race_num,prev_rank,prev_return,prev_sex,prev_start_time,prev_time,prev_time_float,prev_trainer,prev_unknown,prev_weather,prev_weight,prev_winner,prev_year,prev_ymd,race_index,race_num,rank,return,sex,start_time,time,time_float,trainer,unknown,weather,weight,winner,year,ymd,prev_high_rank,high_rank,prev_rank_level,rank_level,prev_popularity_level,prev_speed,prev_speed_level,final_return
529776,0.536172,0.462872,3,56.0,0,23,5,-6.0,0,2000.0,3,html/201008010710.html,9,1,3,45,1,14752,16.3,4.0,2.0,55.0,0.0,27.0,15.0,4.0,0.0,1600.0,5.0,html/200909050807.html,9.0,1.0,10.0,45.0,12.0,14752.0,2.3,1.0,20318.0,7.0,2.0,0.0,1.0,13:10,1:35.1,95.1,7.0,外,0.0,496.0,False,2009.0,20091227.0,36900,10,5.0,0.0,1,15:10,2:03.0,123.0,7,-,0,490.0,False,2010,20100123,1,0,0.0,1.0,0.0,16.824395,16.8,-1.0
529785,0.531796,0.467349,6,57.0,0,11,16,-6.0,0,2000.0,1,html/201003010207.html,9,1,2,59,4,33687,2.1,1.0,6.0,57.0,0.0,14.0,11.0,0.0,1.0,2000.0,5.0,html/201007020412.html,9.0,1.0,10.0,59.0,3.0,33687.0,11.5,3.0,11687.0,12.0,2.0,0.0,1.0,16:01,2:02.5,122.5,85.0,-,0.0,532.0,False,2010.0,20100314.0,36901,7,8.0,0.0,1,13:30,2:01.3,121.3,85,-,1,526.0,False,2010,20100411,1,0,0.0,2.0,1.0,16.326531,16.3,-1.0
529803,0.611704,0.386741,3,54.0,1,3,2,-4.0,1,1400.0,2,html/201805010309.html,9,1,2,32,2,52232,4.7,3.0,3.0,54.0,0.0,13.0,7.0,14.0,0.0,1600.0,4.0,html/201806010409.html,9.0,1.0,4.0,32.0,1.0,52232.0,5.5,2.0,40406.0,9.0,3.0,0.0,0.0,14:25,1:36.3,96.3,34.0,外,0.0,512.0,False,2018.0,20180113.0,36902,9,5.0,0.0,0,14:35,1:22.7,82.7,34,-,1,508.0,False,2018,20180203,1,0,1.0,1.0,0.0,16.614746,16.6,-1.0
529817,0.537237,0.461706,2,54.0,0,2,15,-6.0,0,1400.0,6,html/201908050111.html,3,1,11,24,11,56548,15.8,7.0,2.0,54.0,0.0,1.0,5.0,6.0,0.0,1200.0,5.0,html/201901020610.html,9.0,1.0,7.0,24.0,9.0,56548.0,4.7,2.0,38676.0,10.0,2.0,0.0,0.0,14:50,1:10.9,70.9,3.0,-,0.0,492.0,False,2019.0,20190901.0,36903,11,5.0,0.0,0,15:45,1:21.3,81.3,3,外,0,486.0,False,2019,20191102,1,0,0.0,1.0,0.0,16.925247,16.9,-1.0
529832,0.532413,0.466725,3,54.0,0,27,15,2.0,0,1800.0,6,html/201606020107.html,9,1,8,102,2,678,1.5,1.0,3.0,54.0,0.0,23.0,7.0,-4.0,0.0,1800.0,5.0,html/201606010709.html,9.0,1.0,7.0,67.0,1.0,678.0,5.0,3.0,44779.0,9.0,2.0,0.0,0.0,14:25,1:49.6,109.6,80.0,-,1.0,490.0,False,2016.0,20160123.0,36904,7,3.0,0.0,0,13:30,1:48.9,108.9,80,-,0,492.0,False,2016,20160227,1,1,0.0,1.0,1.0,16.423358,16.4,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661516,0.694349,0.300389,6,57.0,0,5,0,2.0,0,1200.0,3,html/201403020108.html,9,1,5,71,7,5948,3.6,1.0,6.0,57.0,4.0,7.0,2.0,6.0,1.0,1400.0,1.0,html/201405030110.html,9.0,1.0,2.0,22.0,6.0,5948.0,71.6,13.0,45767.0,10.0,4.0,0.0,2.0,15:10,1:26.8,86.8,160.0,-,2.0,472.0,False,2014.0,20140607.0,46073,8,1.0,3.6,2,14:05,1:08.5,68.5,160,-,1,474.0,True,2014,20140705,0,1,1.0,0.0,3.0,16.129032,16.1,3.6
661529,0.529745,0.469492,3,56.0,1,10,1,12.0,0,1700.0,5,html/201603020404.html,9,0,8,32,7,17162,4.6,2.0,3.0,56.0,0.0,19.0,11.0,-2.0,1.0,2100.0,5.0,html/201605030602.html,9.0,0.0,8.0,32.0,6.0,17162.0,2.8,1.0,11559.0,2.0,2.0,0.0,1.0,10:40,2:14.8,134.8,151.0,-,1.0,490.0,False,2016.0,20160619.0,46074,4,3.0,0.0,1,11:40,1:48.3,108.3,151,-,0,502.0,False,2016,20160710,1,1,0.0,1.0,0.0,15.578635,15.6,-1.0
661545,0.537237,0.461706,3,56.0,0,13,15,4.0,1,1600.0,8,html/200704010605.html,9,1,16,58,5,49640,5.9,3.0,3.0,56.0,0.0,28.0,4.0,-4.